In [10]:
pip install -r requirements-Copy1.txt

  Using cached certifi-2023.7.22-py3-none-any.whl.metadata (2.2 kB)
  Using cached ipython-8.17.2-py3-none-any.whl.metadata (6.0 kB)
  Using cached jupyterlab-4.0.8-py3-none-any.whl.metadata (15 kB)
  Using cached jupyterlab_server-2.25.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached mistune-0.8.4-py2.py3-none-any.whl.metadata (8.5 kB)
  Using cached nbconvert-6.5.4-py3-none-any.whl.metadata (6.0 kB)
  Using cached prompt_toolkit-3.0.40-py3-none-any.whl.metadata (6.5 kB)
Using cached certifi-2023.7.22-py3-none-any.whl (158 kB)
Using cached ipython-8.17.2-py3-none-any.whl (808 kB)
Using cached jupyterlab-4.0.8-py3-none-any.whl (9.2 MB)
Using cached jupyterlab_server-2.25.1-py3-none-any.whl (58 kB)
Using cached mistune-0.8.4-py2.py3-none-any.whl (16 kB)
Using cached nbconvert-6.5.4-py3-none-any.whl (563 kB)
Using cached prompt_toolkit-3.0.40-py3-none-any.whl (385 kB)
  Attempting uninstall: mistune
    Found existing installation: mistune 3.0.2
    Uninstalling mistune-3.0.2:
      S

In [20]:
import dotenv
dotenv.load_dotenv()


True

In [2]:
from git import Repo
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language

In [6]:
# Clone
repo_path = "/Users/swaramenon/Documents/llama"
repo = Repo.clone_from("https://github.com/redapt/pyspark-s3-parquet-example", to_path=repo_path)

In [7]:
# Load
loader = GenericLoader.from_filesystem(
    repo_path + "/pyspark-scripts",
    glob="**/*",
    suffixes=[".py"],
    exclude=["**/non-utf8-encoding.py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500),
)
documents = loader.load()
len(documents)

2

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)
texts = python_splitter.split_documents(documents)
len(texts)

3

In [9]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))
retriever = db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 8},
)

In [14]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from llama_index.core import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_community.chat_models import ChatOllama
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate  # instructions to guide LLM
from langchain.chat_models import ChatOpenAI # LLM model
from langchain.schema import StrOutputParser

# llm = ChatOllama(model="llama3", temperature=0)
llm = ChatOllama(model="llama3:instruct", temperature=0)

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
doc_prompt_custom = PromptTemplate.from_template(template)

doc_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | doc_prompt_custom
    | llm
    | StrOutputParser()
)

In [16]:
doc_chain.invoke("Explain the repo")

Number of requested results 20 is greater than number of elements in index 3, updating n_results = 3


'The repository appears to be a collection of Python scripts that demonstrate how to use PySpark, a Python API for Apache Spark, to load and query Parquet-formatted files. The scripts are designed to run on both AWS EMR (a cloud-based Hadoop environment) and locally.\n\nThe scripts import the necessary modules, including `pyspark` and `SQLContext`, and then use these modules to read in a Parquet file, register it as an in-memory temporary table, and execute SQL queries against that table. The results of these queries are then printed out.\n\nOne script is configured to run on AWS EMR and pulls the Parquet file from S3, while the other script is designed to run locally and loads the Parquet file from the local file system.\n\nThanks for asking!'

In [17]:
doc_chain.invoke("What is the input to the read?")

Number of requested results 20 is greater than number of elements in index 3, updating n_results = 3


'The input to the `read.parquet` method in this script is a string that specifies the location of the .parquet file. In the AWS EMR version, it\'s "s3://jon-parquet-format/nation.plain.parquet", and in the local version, it\'s \'../test-data/nation.plain.parquet\'. Thanks for asking!'

In [18]:
doc_chain.invoke("What is the output to the write?")

Number of requested results 20 is greater than number of elements in index 3, updating n_results = 3


"I don't know. The provided code snippets are Python scripts that use PySpark to load a Parquet file and run SQL queries against it, but they do not specify what the output should be or how it should be written."

In [19]:
doc_chain.invoke("Is there any data transformation?")

Number of requested results 20 is greater than number of elements in index 3, updating n_results = 3


'No, there is no explicit data transformation in the provided code. The script loads a Parquet file into an RDD DataFrame and then runs SQL queries against it without modifying the data. Thanks for asking!'